In [2]:
from ultralytics import YOLO
import cv2
import math
import matplotlib.pyplot as plt
import pytesseract
import easyocr
import numpy as np

In [ ]:
# Cargar modelo
model = YOLO('yolov8n.yaml') # Construir nuevo modelo desde yaml

#results = model.train(data='spanish_license_plates_dataset/data.yaml', epochs=50, imgsz=640)


In [ ]:
def read_license_plate_easyocr(license_plate_crop):
    # Funcion para leer obtener como texto una matricula en una imagen

    # Carga del modelo de lengua Easyocr
    reader = easyocr.Reader(['en'], gpu=False) 

    # Reconocimiento de una imagen
    total_license = ""
    results = reader.readtext(license_plate_crop, min_size=120)
    for result in results:
        bbox, text, score = result
        text = text.upper().replace(' ', '')
        total_license += text
    return(total_license)


def read_license_plate_pytesseract(license_plate_crop):
    # Funcion para leer obtener como texto una matricula en una imagen

    # Carga del modelo de lengua Pytesseract
    pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract'

    # Lenguajes disponibles
    pytesseract.get_languages(config='')

    #Aplica reconocedor a imagen cargada
    return pytesseract.image_to_string(license_plate_crop)


In [ ]:
# Carga el modelo con los pesos de la mejor epoca
bestModel = YOLO('runs/detect/train6/weights/best.pt')
classNames = ['matricula'] 

image_path = 'media/imagetest6.jpg'
img = cv2.imread(image_path)


img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

results = bestModel(img, stream=True)

for r in results:
    boxes = r.boxes

    # Matriculas detectadas
    for box in boxes:
        x1, y1, x2, y2 = box.xyxy[0]
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
        
        # Confianza
        confidence = math.ceil((box.conf[0]*100))/100
        print("Confidence --->",confidence)           

        # Clase
        cls = int(box.cls[0])
        print("Class name -->", classNames[cls])

        # Convierte identificador numérico de clase a un color RGB
        escala = int((cls / len(classNames)) * 255 * 3)
        if escala >= 255*2:
            R = 255
            G = 255
            B = escala - 255*2
        else:
            if escala >= 255:
                R = 255
                G = escala - 255
                B = 0
            else:
                R = escala
                G = 0
                B = 0

        # Dibuja el contenedor y clase
        cv2.rectangle(img, (x1-10, y1-10), (x2+10, y2+10), (0, 0, 255), 4)

        # Imagen recortada de la matricula
        license_plate_crop = img[y1 - 10:y2 + 10, x1 - 10:x2 + 10]

        license_text = read_license_plate_easyocr(license_plate_crop)
        cv2.putText(img, license_text, [x1 -10, y1 -10], cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, B), 3)
        print(license_text)

plt.axis("off")
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB)) 
plt.show()







In [ ]:
# Carga el modelo con los pesos de la mejor epoca
bestModel = YOLO('runs/detect/train6/weights/best.pt')
classNames = ['matricula']

# Carga el video
cap = cv2.VideoCapture('media/prueba.mp4')  
while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        break

    results = bestModel(frame, stream=True)

    for r in results:
        boxes = r.boxes

        # Matriculas detectadas
        for box in boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])

            cv2.rectangle(frame, (x1 - 10, y1 - 10), (x2 + 10, y2 + 10), (0, 0, 255), 4)
            license_text = read_license_plate_pytesseract(license_plate_crop)
            cv2.putText(img, license_text, [x1 -10, y1 -10], cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, B), 3)

    cv2.imshow('Video', frame)

    # Pulsar 'q' para salir del video
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam
cap.release()
cv2.destroyAllWindows()

In [4]:
# Carga el modelo con los pesos de la mejor epoca
bestModel = YOLO('runs/detect/train6/weights/best.pt')
classNames = ['matricula']
vid_path = "media/prueba.mp4"

for results in bestModel.track(source=vid_path, conf=0.3, iou=0.5, persist=True, show=True):
    frame = results.orig_img

    cv2.imshow("yolov8", frame)

    if(cv2.waitKey(30) & 0xFF) == 27:
        break

cv2.destroyAllWindows()



WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (1/698) c:\Users\thewi\vc_practicas\p5\entrega5\P5-Reconocimiento-De-Matriculas\media\prueba.mp4: 384x640 2 plates, 71.0ms
video 1/1 (2/698) c:\Users\thewi\vc_practicas\p5\entrega5\P5-Reconocimiento-De-Matriculas\media\prueba.mp4: 384x640 2 plates, 63.0ms
video 1/1 (3/698) c:\Users\thewi\vc_practicas\p5\entrega5\P5-Reconocimiento-De-Matriculas\media\prueba.mp4: 384x640 2 plates, 56.0ms
video 1/1 (4/698) c:\Users\thewi\vc_practicas\p5\entrega5\P

KeyboardInterrupt: 